### General information referring to some of the code

**patsy**
*Generating design matrices*
We include the intercept once in the model via the intercept parameter, to prevent having additional '1' columns in basis functions include_intercept is set to False and automatic creation of '1' column of patsy is prevented by '-1' in the formula.

*Passing design matrices to PyMc/ Stan*
For the spline regression we generate our design matrices using patsy. This returns an object of the class 'DesignMatrix' which contains additional metadata comparede to a numpy array. However in comparioson to numpy arrays, those objects of the DesignMatrix clss do not support pickling.
Therefore we convert the matrices to numpy arrays by np.asarray() - essentially dropping the metadata - before passing them to the model.
THe additional argument order="F" is used to have the array in Fortran order which is preferred by PyMC3 and Stan.

**Understanding posterior predictives and cross validation**
(1) We specify a model and incorporate some prior belief for the model parameters.
(2) Using the training data and the MCMC algorithm we update our prior beliefs to better represent the data we obeserve. Thereby we obtain posterior distributions for our model parameters.
(3) With new data (our evaluation set), we can verify our model. Notice, that the posterior distributions of all parameters we defined in our model remain unchanged. In our specific example only mu is individual for each parameter as it is dependent on the covariates of a observation. This explains how we obtain a different mu for every obeservation if the covariates aren't identical. To verify the model we draw samples from the posterior distribution of each observation and calculate the CRPS.
(how are the posterior predictive samples obtained: we draw a set of paramters based on their underlying posterior distribution and use those to draw one sample. This is repeated several times to reflect the uncertainty within  the posteriors for this obeservation.

**Implementing cross validation**
PyStan: Add corresponding code to the 'generate quantities block' to generate the posterior predictive samples based on obtained posteriors and new data
PyMc: Define covariates as "theano.shared" variables before feeding them into the model and integrate evaluation data via the "set_value()" function. Afterwards, pm.sample_posterior_predictive() will generate predictives for the evaluation data only. 

**Transforming data**
According to Bretzger, Lang (2004) it is desirable to standardize out target variable as "the amount of smoothness allowed by a particular prior specification depends (weakly) on the scale of the responses". However, initially in Stan this rose an error originating from working with a negative binomial distribution. Further inspection is necessary.

**Defining knots for splines**
So far we use equally-spaced knots for our spline regression. As our data is zero-inflated and overdispersed this leads to having the majority of the data within a minorty of the knots. Quantile-based knots are therefore a valid option. However, the P-Splines definition through a penalty matrix is not compatible with quantile-based knots. For an integration of quantile-based knots further investigation is necessary.

## I. Prerequisites

**1. Import Packages**

In [1]:
import arviz as az
import numpy as np
import pandas as pd
import time

from data_gathering import gather_data_actuals, gather_data_features
from data_preparation import preprocess_data, generate_knots, generate_spline_design_matrix
from data_exploration import calculate_data_characteristics
from data_modelling_bayes import calculate_crps, generate_penalty_matrix, generate_all_dictionaries, ml_flow_tracking, count_divergences
from help_functions import MLFlowServer
from importlib import reload


In [2]:
# Reload modules to reflect changes in imported files
import data_preparation
reload(data_preparation)
from data_preparation import generate_spline_design_matrix


In [3]:
pymc = False
stan = True
example_export = False
start_server = True

**PyMc specific packages**

In [4]:
if pymc:
    import pymc as pm

**PyStan specific packages**

In [5]:
if stan:
    import pystan

In [31]:
if start_server:
    # Start mlflow server: Necessary before every first mlflow_logging
    server = MLFlowServer(experiment_name="bayesian_models")
    server.start()

KeyboardInterrupt: 

In [29]:
server.stop()

**2. Define functions**

**3. Set style and generate random seed**

In [7]:
az.style.use("arviz-darkgrid")
RANDOM_SEED = 58

## II. Data Preparation

### 1. Load data

In [8]:
data_cm_actual_2018, data_cm_actual_2019, data_cm_actual_2020, data_cm_actual_2021, data_cm_actual_allyears \
    = gather_data_actuals()
# Load features data
data_cm_features_2017, data_cm_features_2018, data_cm_features_2019, data_cm_features_2020, data_cm_features_allyears \
    = gather_data_features()

### 2. Define model specifications corresponding to mlflow log

**Data used in model**

In [9]:
predictors = ['ged_sb_tlag_1', 'ged_sb_tsum_24']
target_variable = ['ged_sb']
country_name = 'Syria'
df_Y_non_std_X_std, df_Y_std_X_std, df_Y_non_std_X_std_log, df_Y_std_X_std_log = (preprocess_data(data_cm_features_allyears, predictors, target_variable, country_name))
df = df_Y_non_std_X_std

C:\Users\Uwe Drauz\anaconda3\envs\bachelor_thesis_pystan_clean\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
# Note: Prelimanary definition of evaluation data till better integration of cross validation
# Take last 24 obeservations of training data a
df_eval = df.iloc[-24:]
# Select 3rd till 14th observation (equals year 2019 here) as evaluation data
df_eval = df_eval.iloc[3:14]
# Cut the last 24 observations from the training data
df = df.iloc[:-24]

In [11]:
dataX1 = df.ged_sb_tlag_1.to_numpy()
dataX2 = df.ged_sb_tsum_24.to_numpy()
dataY = df.ged_sb.to_numpy()
dataY_df = pd.DataFrame(df["ged_sb"])
data_characteristics_train = calculate_data_characteristics(df)

In [12]:
dataX1_eval = df_eval.ged_sb_tlag_1.to_numpy()
dataX2_eval = df_eval.ged_sb_tsum_24.to_numpy()
dataY_eval = df_eval.ged_sb.to_numpy()
dataY_eval_df = pd.DataFrame(df_eval["ged_sb"])
data_characteristics_eval = calculate_data_characteristics(df_eval)

In [13]:
data_transformation = "target variable not standardized, input data standardized"

In [ ]:
# Export to test in R
if example_export:
    df.to_parquet(fr'C:\Users\Uwe Drauz\Documents\bachelor_thesis_local\personal_competition_data\fatality_data_{country_name}.parquet')


### 3. Model description

In [14]:
model_name_gauss = 'bayesian_regression'
model_name_nb_half_normal = 'bayesian_regression'
model_name_nb_gamma = 'bayesian_regression'
model_name_splines = 'bayesian_spline_regression'
model_name_penalized_splines_grw = 'bayesian_penalized_spline_regression_gaussian_random_walk'
model_name_penalized_splines_penalty = 'bayesian_penalized_spline_regression_penalty_potential'

In [15]:
prior_specifications_gauss = {
    "intercept": "Normal",
    "slopesX1": "Normal", 
    "slopesX2": "Normal", 
    "likelihood": "Gaussian",
    "sigma": "Normal", 
}
prior_specifications_nb_half_normal = {
    "intercept": "Normal",
    "slopesX1": "Normal", 
    "slopesX2": "Normal", 
    "likelihood": "NegativeBinomial",
    "alpha": "HalfNormal", 
}
prior_specifications_nb_gamma = {
    "intercept": "Normal",
    "slopesX1": "Normal", 
    "slopesX2": "Normal", 
    "likelihood": "NegativeBinomial",
    "alpha": "Gamma", 
}
prior_specifications_splines = {
    "intercept": "Normal",
    "slopesX1": "Normal", 
    "slopesX2": "Normal", 
    "likelihood": "NegativeBinomial",
    "alpha": "Gamma", 
}
prior_specifications_penalized_splines_grw = {
    "intercept": "Normal",
    "inital_distribution": "Normal",
    "slopesX1": "GaussianRandomWalk", 
    "tauX1": "InverseGamma",
    "slopesX2": "GaussianRandomWalk", 
    "tauX2": "InverseGamma",
    "likelihood": "NegativeBinomial",
    "alpha": "Gamma", 
}
prior_specifications_penalized_splines_penalty = {
    "intercept": "Normal",
    "slopesX1_0": "Normal",
    "slopesX1_rest": "Normal", 
    "tauX1": "InverseGamma",
    "slopesX2_0": "Normal",
    "slopesX2_rest": "Normal", 
    "tauX2": "InverseGamma",
    "likelihood": "NegativeBinomial",
    "alpha": "Gamma", 
}

### 4. Model hyperparameters

In [16]:
# PARAMETERS RELATED TO SPLINES
num_knots_X1 = 10
num_knots_X2 = 10
spline_degree = 3
no_spline_coefficients_X1 = num_knots_X1 + spline_degree
no_spline_coefficients_X2 = num_knots_X2 + spline_degree

In [17]:
# name scheme: y ~ dist1(z), x ~ dist2(y)
# opt(component name) + dist1 + z + dist2 + y
# PARAMETERS RELATED TO PRIORS
# Define variables for tau priors
tauX1_ig_alpha_gaussian_sigma = tauX2_ig_alpha_gaussian_sigma = 1
tauX1_ig_beta_gaussian_sigma = tauX2_ig_beta_gaussian_sigma = 0.005

# Define variables for alpha prior of gamma distribution of alpha of negative binomial distribution
gamma_alpha_nb_alpha = 0.1
gamma_beta_nb_alpha = 0.1

# Define variables for prior distribution of intercept
intercept_gaussian_mu = 0
intercept_gaussian_sigma = 5

# Define standard deviation for Half Normal distribution of  alpha parameter of likelihood negative binomial distribution
hgaussian_sigma_nb_alpha = 2.5

# Define standard deviation for Normal distribution of sigma of likelihood gaussian distribution
gaussian_sigma_gaussian_sigma = 2.5

# Define parameters for gaussian distribution of regression coefficients
beta_gaussian_mu = 0.0
beta_gaussian_sigma = 1.0

# Define paramters for half gaussian distribution of intercepts
intercept_hgaussian_mu = 0
intercept_hgaussian_sigma = 1

# Define parameters for beta diffuse prior
diffuse_prior_beta_mu = 0
diffuse_prior_beta_sigma = 100

# Define sigma of half gaussian distribution of sigma for gaussian likelihood
hgaussian_sigma_gaussian_sigma = 2.5

### 5. Sampling hyperparameters

In [18]:
# Define number of tuning iterations
tune_default = 2000
# Define number of sampling iterations
draws_default = 1000
# Define target acceptance rate
target_accept_default = 0.95
# Define number of chains
no_chains_default = 4
# Define number of posterior predictive samples
no_ppc_samples_default = 500

In [19]:
# Define dictionary for implicit handling of data in PyMc
COORDS_gauss = {"predictors": ['ged_sb_tlag_1', 'ged_sb_tsum_24'], "obs_idx": df.index}
COORDS_nb_half_normal = {"predictors": ['ged_sb_tlag_1', 'ged_sb_tsum_24'], "obs_idx": df.index}
COORDS_nb_gamma = {"predictors": ['ged_sb_tlag_1', 'ged_sb_tsum_24'], "obs_idx": df.index}
COORDS_splines = {"obs_idx": df.index, "splines_x1": np.arange(no_spline_coefficients_X1), "splines_x2": np.arange(no_spline_coefficients_X2)}
COORDS_penalized_splines_rw = {"obs_idx": df.index, "splines_x1": no_spline_coefficients_X1, "splines_x2": no_spline_coefficients_X2}
COORDS_penalized_splines_penalty = \
    {"obs_idx": df.index, "splines_x1_rest": np.arange(no_spline_coefficients_X1 -1), "splines_x2_rest": np.arange(no_spline_coefficients_X2 -1)}

### 6. Calculate design matrices for implementation of splines

In [20]:
# We define the knot list such that we'll have num_knots knot inbetween the boundaries of the data
knot_list_X1 = generate_knots(dataX1, num_knots_X1)
knot_list_X2 = generate_knots(dataX2, num_knots_X2)

# Define the basis functions
basis_X1 = generate_spline_design_matrix(dataX1, knot_list_X1, spline_degree)
basis_X2 = generate_spline_design_matrix(dataX2, knot_list_X2, spline_degree)

# Define the penalty matrices
K_X1 = generate_penalty_matrix(basis_X1.shape[1])
K_X2 = generate_penalty_matrix(basis_X2.shape[1])

if basis_X1.shape[1] != no_spline_coefficients_X1:
    print("Number of coefficients as per basis function of regressor 1 did not match predefined number of coefficients in spline. Changed it by considering dimensions of basis design matrix")
    no_spline_coefficients_X1 = basis_X1.shape[1]
if basis_X2.shape[1] != no_spline_coefficients_X2:
    print("Number of coefficients as per basis function of regressor 2 did not match predefined number of coefficients in spline. Changed it by considering dimensions of basis design matrix")
    no_spline_coefficients_X2 = basis_X2.shape[1]

# Define the design matrices for the evaluation data
knot_list_X1_eval = generate_knots(dataX1_eval, num_knots_X1)
knot_list_X2_eval = generate_knots(dataX2_eval, num_knots_X2)

basis_X1_eval = generate_spline_design_matrix(dataX1_eval, knot_list_X1_eval, spline_degree)
basis_X2_eval = generate_spline_design_matrix(dataX2_eval, knot_list_X2_eval, spline_degree)


## III.1 PyMc Model Building

### 1. Build Bayesian model with Gaussian distribution

In [ ]:
start= time.time()
with pm.Model(coords=COORDS_gauss) as bayesian_model_gauss:
    # Define the priors for regressors and standard deviation
    intercept = pm.Normal('intercept', intercept_gaussian_mu, intercept_gaussian_sigma)
    beta = pm.Normal("slopes", beta_gaussian_mu, beta_gaussian_sigma, dims="predictors")
    sigma = pm.HalfNormal("sigma", hgaussian_sigma_gaussian_sigma)

    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
    Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")

    # Specify mean of gaussian distribution
    mu = intercept +  beta[0] * X1 + beta[1] * X2
    # mean for target variable
    obs = pm.Normal("obs", mu=mu, sigma=sigma, observed=Y, dims="obs_idx")

    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_gauss = pm.sample(draws=draws_default, tune=tune_default, random_seed=RANDOM_SEED, target_accept=target_accept_default)

sampling_time_gauss = time.time() - start

In [ ]:
with bayesian_model_gauss:
    ppc_prior_gauss = pm.sample_prior_predictive(no_ppc_samples_default, random_seed=RANDOM_SEED)
    ppc_posterior_gauss = pm.sample_posterior_predictive(idata_bayesian_gauss, extend_inferencedata=True, random_seed=RANDOM_SEED)

In [ ]:
crps_gauss, crps_average_gauss = calculate_crps(idata_bayesian_gauss, actuals=dataY_df, posterior_predictive='obs');

In [ ]:
# Create dictionary of parameters used in the model:
divergences_gauss = count_divergences(idata_bayesian_gauss)
all_dictionaries_gauss = generate_all_dictionaries(
    training_data=df, 
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train, 
    data_characteristics_eval=data_characteristics_eval,
    data_transformation=data_transformation, 
    country_name=country_name, 
    predictors=predictors, 
    model_name=model_name_gauss, 
    prior_specifications=prior_specifications_gauss,
    intercept_hyperparameters= (intercept_gaussian_mu, intercept_gaussian_sigma),
    beta_hyperparameters=(beta_gaussian_mu, beta_gaussian_sigma),
    sigma_gaussian_hyperparameters=hgaussian_sigma_gaussian_sigma,
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_gauss,
    sampling_time=sampling_time_gauss, 
    idata=idata_bayesian_gauss,
    crps_score_train=crps_average_gauss,
    crps_score_eval=crps_average_gauss)
ml_flow_tracking(**all_dictionaries_gauss)

### 2. Build Bayesian model with Negative Binomial distribution (alpha HalfNormal distributed)

In [ ]:
# Build the model
start = time.time()
with pm.Model(coords=COORDS_nb_half_normal) as bayesian_model_nb_half_normal:
    # Define the priors for regressors and  negative binomial over-dispersion parameter
    intercept = pm.Normal("intercept", mu=intercept_gaussian_mu, sigma=intercept_gaussian_sigma)
    beta = pm.Normal("slopes", beta_gaussian_mu, beta_gaussian_sigma, dims="predictors")
    alpha = pm.HalfNormal('alpha', sigma=hgaussian_sigma_nb_alpha)

    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
    Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")

    # Note: Possibly an option to change to pm.Deterministic
    # Define mean of negative binomial distribution
    mu = pm.math.exp(intercept + beta[0] * X1 + beta[1] * X2)

    # Define the likelihood
    obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

    # Note: Optionally define cores and chains
    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_nb_half_normal = pm.sample(draws=draws_default, tune=tune_default, random_seed=RANDOM_SEED, target_accept=target_accept_default)

sampling_time_nb_half_normal = time.time() - start

In [ ]:
try:
    with bayesian_model_nb_half_normal:
        ppc_posterior_nb_half_normal = pm.sample_posterior_predictive(idata_bayesian_nb_half_normal, extend_inferencedata=True, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)
try:
    with bayesian_model_nb_half_normal:
        ppc_prior_nb_half_normal = pm.sample_prior_predictive(no_ppc_samples_default, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)

In [ ]:
crps_nb_half_normal, crps_average_nb_half_normal = calculate_crps(idata_bayesian_nb_half_normal, actuals=dataY_df, posterior_predictive='obs');

In [ ]:
# Create dictionary of parameters used in the model:
divergences_nb_half_normal = count_divergences(idata_bayesian_nb_half_normal)
all_dictionaries_nb_half_normal = generate_all_dictionaries(
    training_data=df, 
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train, 
    data_characteristics_eval=data_characteristics_eval,
    data_transformation=data_transformation, 
    country_name=country_name, 
    predictors=predictors, 
    model_name=model_name_nb_half_normal, 
    prior_specifications=prior_specifications_nb_half_normal,
    alpha_nb_hyperparameters=hgaussian_sigma_nb_alpha,
    intercept_hyperparameters= (intercept_gaussian_mu, intercept_gaussian_sigma),
    beta_hyperparameters=(beta_gaussian_mu, beta_gaussian_sigma),
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_nb_half_normal,
    sampling_time=sampling_time_nb_half_normal, 
    idata=idata_bayesian_nb_half_normal,
    crps_score_train=crps_average_nb_half_normal,
    crps_score_eval=crps_average_nb_half_normal)
ml_flow_tracking(**all_dictionaries_nb_half_normal)

### 3. Build Bayesian model with Negative Binomial distribution (alpha Gamma distributed)

In [ ]:
# Build the model
start = time.time()
with pm.Model(coords=COORDS_nb_gamma) as bayesian_model_nb_gamma:
    # Define the priors for regressors and  negative binomial over-dispersion parameter
    intercept = pm.Normal("intercept", mu=intercept_gaussian_mu, sigma=intercept_gaussian_sigma)
    beta = pm.Normal("slopes", beta_gaussian_mu, beta_gaussian_sigma, dims="predictors")
    alpha = pm.Gamma('alpha', gamma_alpha_nb_alpha, gamma_beta_nb_alpha)

    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
    Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")

    # Note: Possibly an option to change to pm.Deterministic
    # Define mean of negative binomial distribution
    mu = pm.math.exp(intercept + beta[0] * X1 + beta[1] * X2)

    # Define the likelihood
    obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

    # Note: Optionally define cores and chains
    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_nb_gamma = pm.sample(draws=draws_default, tune=tune_default, random_seed=RANDOM_SEED, target_accept=target_accept_default)

sampling_time_nb_gamma = time.time() - start

In [ ]:
try:
    with bayesian_model_nb_gamma:
        ppc_posterior_nb_gamma = pm.sample_posterior_predictive(idata_bayesian_nb_gamma, extend_inferencedata=True, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)
try:
    with bayesian_model_nb_gamma:
        ppc_prior_nb_gamma = pm.sample_prior_predictive(no_ppc_samples_default, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)

In [ ]:
crps_nb_gamma, crps_average_nb_gamma = calculate_crps(idata_bayesian_nb_gamma, actuals=dataY_df, posterior_predictive='obs');

In [ ]:
# Create dictionary of parameters used in the model:
divergences_nb_gamma = count_divergences(idata_bayesian_nb_gamma)
all_dictionaries_nb_gamma = generate_all_dictionaries(
    training_data=df, 
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train, 
    data_characteristics_eval=data_characteristics_eval,
    data_transformation=data_transformation,
    country_name=country_name, 
    predictors=predictors, 
    model_name=model_name_nb_gamma, 
    prior_specifications=prior_specifications_nb_gamma,
    alpha_nb_hyperparameters=(gamma_alpha_nb_alpha, gamma_beta_nb_alpha),
    intercept_hyperparameters= (intercept_gaussian_mu, intercept_gaussian_sigma),
    beta_hyperparameters=(beta_gaussian_mu, beta_gaussian_sigma),
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_nb_gamma,
    sampling_time=sampling_time_nb_gamma, 
    idata=idata_bayesian_nb_gamma,
    crps_score_train=crps_average_nb_gamma,
    crps_score_eval=crps_average_nb_gamma)
ml_flow_tracking(**all_dictionaries_nb_gamma)

### 4. Build Bayesian model with Negative Binomial distribution (alpha Gamma distributed) and regression splines (B-Splines)

In [ ]:
# Build the model
start = time.time()
with pm.Model(coords=COORDS_splines) as bayesian_model_nb_gamma_splines:
    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
    Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")


    # Define the priors for regressors and  negative binomial over-dispersion parameter
    intercept = pm.Normal("intercept", mu=intercept_hgaussian_mu, sigma=intercept_hgaussian_sigma)
    beta_X1 = pm.Normal("slopesX1", beta_gaussian_mu, beta_gaussian_sigma, dims="splines_x1")
    beta_X2 = pm.Normal("slopesX2", beta_gaussian_mu, beta_gaussian_sigma, dims="splines_x2")
    alpha = pm.Gamma('alpha', gamma_alpha_nb_alpha, gamma_beta_nb_alpha)
    
    # Define mean of negative binomial distribution
    mu = pm.math.exp(intercept + pm.math.dot(np.asarray(basis_X1, order="F"), beta_X1.T) + pm.math.dot(np.asarray(basis_X2, order="F"), beta_X2.T))

    # Define the likelihood
    obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

    # Note: Optionally define cores and chains
    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_nb_gamma_splines = pm.sample(draws=draws_default, tune=tune_default, random_seed=RANDOM_SEED, target_accept=target_accept_default)

sampling_time_nb_gamma_splines = time.time() - start

In [ ]:
try:
    with bayesian_model_nb_gamma_splines:
        ppc_posterior_nb_gamma_splines = pm.sample_posterior_predictive(idata_bayesian_nb_gamma_splines, extend_inferencedata=True, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)
try:
    with bayesian_model_nb_gamma_splines:
        ppc_prior_nb_gamma_splines = pm.sample_prior_predictive(no_ppc_samples_default, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)

In [ ]:
crps_nb_gamma_splines, crps_average_nb_gamma_splines = calculate_crps(idata_bayesian_nb_gamma_splines, actuals=dataY_df, posterior_predictive='obs');

In [ ]:
# Create dictionary of parameters used in the model:
divergences_nb_gamma_splines = count_divergences(idata_bayesian_nb_gamma_splines)
all_dictionaries_splines = generate_all_dictionaries(
    training_data=df, 
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train, 
    data_characteristics_eval=data_characteristics_eval,
    data_transformation=data_transformation, 
    country_name=country_name, 
    predictors=predictors, 
    model_name=model_name_splines, 
    prior_specifications=prior_specifications_splines,
    alpha_nb_hyperparameters=(gamma_alpha_nb_alpha, gamma_beta_nb_alpha),
    intercept_hyperparameters= (intercept_gaussian_mu, intercept_gaussian_sigma),
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_nb_gamma_splines,
    sampling_time=sampling_time_nb_gamma_splines, 
    idata=idata_bayesian_nb_gamma_splines,
    crps_score_train=crps_average_nb_gamma_splines,
    crps_score_eval=crps_average_nb_gamma_splines)
ml_flow_tracking(**all_dictionaries_splines)

### 5. Build Bayesian model with Negative Binomial distribution (alpha Gamma distributed) and penalized regression splines with gaussin random walk

In [ ]:
# NOTE: WIP: Implementation of penalized regression splines still to be figured out, Version with Gaussian Random Walk
# Track the time it takes o sample model
start_time = time.time()
# Build the model with random walk
with pm.Model(coords=COORDS_splines) as bayesian_model_nb_gamma_penalized_splines_grw:
    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
    Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")


    # Define the priors for regressors and  negative binomial over-dispersion parameter
    intercept = pm.Normal("intercept", mu=intercept_gaussian_mu, sigma=intercept_gaussian_sigma)

    # Define first order random walk to incorporate regularization
    tau_X1 = pm.InverseGamma("tauX1", alpha=tauX1_ig_alpha_gaussian_sigma, beta=tauX1_ig_beta_gaussian_sigma)
    tau_X2 = pm.InverseGamma("tauX2", alpha=tauX2_ig_alpha_gaussian_sigma, beta=tauX2_ig_beta_gaussian_sigma)

    # Define the initial distribution as diffuse prior for the random walk
    initial_distribution_X1 = pm.Normal.dist(mu=diffuse_prior_beta_mu, sigma=diffuse_prior_beta_sigma)
    initial_distribution_X2 = pm.Normal.dist(mu=diffuse_prior_beta_mu, sigma=diffuse_prior_beta_sigma)
    
    # Note: Defining the coefficients in a gaussian random walk should incorporate the penalization of differences of consecutive coefficients
    beta_X1 = pm.GaussianRandomWalk("slopesX1", sigma=tau_X1, shape=basis_X1.shape[1], init_dist=initial_distribution_X1)
    beta_X2 = pm.GaussianRandomWalk("slopesX2", sigma=tau_X2, shape=basis_X2.shape[1], init_dist=initial_distribution_X2)
    alpha = pm.Gamma('alpha', gamma_alpha_nb_alpha, gamma_beta_nb_alpha)

    # Define mean of negative binomial distribution
    # Note: We define the mean of the assumed negative binomial as a linear combination of the basis functions - transformed by a link function to ensure numerical stability
    mu = pm.math.exp(intercept + pm.math.dot(np.asarray(basis_X1, order="F"), beta_X1.T) + pm.math.dot(np.asarray(basis_X2, order="F"), beta_X2.T))

    # Define the likelihood
    # Note: mu is of dimension [N,1] and weighted against the observations in the sampling process
    obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_nb_gamma_penalized_splines_grw = pm.sample(draws=draws_default, tune=tune_default, random_seed=RANDOM_SEED, target_accept=target_accept_default, chains=no_chains_default)
    
sampling_time_psplines_grw = time.time() - start_time

In [ ]:
try:
    with bayesian_model_nb_gamma_penalized_splines_grw:
        ppc_posterior_nb_gamma_penalized_splines = pm.sample_posterior_predictive(idata_bayesian_nb_gamma_penalized_splines_grw, extend_inferencedata=True, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)
try:
    with bayesian_model_nb_gamma_penalized_splines_grw:
        ppc_prior_nb_gamma_penalized_splines = pm.sample_prior_predictive(no_ppc_samples_default, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)

In [ ]:
# Calculate CRPS
crps_nb_gamma_penalized_splines_grw, crps_average_nb_gamma_penalized_splines_grw = calculate_crps(idata_bayesian_nb_gamma_penalized_splines_grw, dataY_df, posterior_predictive='obs')

In [ ]:
# Create dictionary of parameters used in the model:
divergences_splines = count_divergences(idata_bayesian_nb_gamma_splines)
all_dictionaries_penalized_splines_grw = generate_all_dictionaries(
    training_data=df, 
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train, 
    data_characteristics_eval=data_characteristics_eval,
    data_transformation=data_transformation, 
    country_name=country_name, 
    predictors=predictors, 
    model_name=model_name_penalized_splines_grw, 
    prior_specifications=prior_specifications_splines,
    alpha_nb_hyperparameters=(gamma_alpha_nb_alpha, gamma_beta_nb_alpha),
    intercept_hyperparameters= (intercept_gaussian_mu, intercept_gaussian_sigma),
    no_spline_coefficients_per_regressor=[no_spline_coefficients_X1, no_spline_coefficients_X2],
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_splines,
    sampling_time=sampling_time_psplines_grw, 
    idata=idata_bayesian_nb_gamma_splines,
    crps_score_train=crps_average_nb_gamma_splines,
    crps_score_eval=crps_average_nb_gamma_splines)
ml_flow_tracking(**all_dictionaries_penalized_splines_grw)

### 6. Build Bayesian model with Negative Binomial distribution (alpha Gamma distributed) and penalized regression splines with penalty potential

In [ ]:
# Track sampling time
start_time = time.time()
# NOTE: WIP: Implementation of penalized regression splines still to be figured out, Version with Penalty deduction in likeihood

with pm.Model(coords=COORDS_penalized_splines_penalty) as bayesian_model_nb_gamma_penalized_splines_penalty:
    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
    Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")

    # Define the priors for regressors and negative binomial over-dispersion parameter
    intercept = pm.Normal("intercept", mu=intercept_gaussian_mu, sigma=intercept_gaussian_sigma)

    # Define first order random walk to incorporate regularization
    tau_X1 = pm.InverseGamma("tauX1", alpha=tauX1_ig_alpha_gaussian_sigma, beta=tauX1_ig_beta_gaussian_sigma)
    tau_X2 = pm.InverseGamma("tauX2", alpha=tauX2_ig_alpha_gaussian_sigma, beta=tauX2_ig_beta_gaussian_sigma)

    # Create deterministic variables for squared taus
    tau_X1_squared = pm.Deterministic("tau_X1_squared", tau_X1 ** 2)
    tau_X2_squared = pm.Deterministic("tau_X2_squared", tau_X2 ** 2)
    
    # Note: If this does reflect our intended model remains to be questioned
    # Introduce diffuse priors for the first regression coefficients of the splines with broad gaussian distributions
    beta_X1_0 = pm.Normal("slopesX1_0", diffuse_prior_beta_mu, diffuse_prior_beta_sigma)
    beta_X2_0 = pm.Normal("slopesX2_0", diffuse_prior_beta_mu, diffuse_prior_beta_sigma)
    
    # Define remaining betas as normal variables with standard deviation tau
    beta_X1_rest = pm.Normal("slopesX1_rest", beta_gaussian_mu, tau_X1, dims="splines_x1_rest")
    beta_X2_rest = pm.Normal("slopesX2_rest", beta_gaussian_mu, tau_X2, dims="splines_x2_rest")
    
    # Concatenate the flat priors to the beginning of the coefficients
    beta_X1 = pm.Deterministic("slopesX1", pm.math.concatenate([[beta_X1_0], beta_X1_rest]))
    beta_X2 = pm.Deterministic("slopesX2", pm.math.concatenate([[beta_X2_0], beta_X2_rest]))


    alpha = pm.Gamma('alpha', gamma_alpha_nb_alpha, gamma_beta_nb_alpha)

    # Define mean of negative binomial distribution
    mu = pm.math.exp(intercept + pm.math.dot(np.asarray(basis_X1, order="F"), beta_X1.T) + pm.math.dot(np.asarray(basis_X2, order="F"), beta_X2.T))

    # Define the likelihood
    obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

    # Define the penalty term and incorporate it using pm.Potential
    penalty_term_X1 = pm.math.dot(beta_X1.T, pm.math.dot(K_X1, beta_X1)) / (2 * tau_X1_squared)
    penalty_X1 = pm.Potential("penalty_X1", -penalty_term_X1)

    penalty_term_X2 = pm.math.dot(beta_X2.T, pm.math.dot(K_X2, beta_X2)) / (2 * tau_X2_squared)
    penalty_X2 = pm.Potential("penalty_X2", -penalty_term_X2)

    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_nb_gamma_penalized_splines_penalty = pm.sample(draws=draws_default, tune=tune_default, random_seed=RANDOM_SEED, target_accept=target_accept_default, chains=no_chains_default)

sampling_time_psplines_penalty = time.time() - start_time

In [ ]:
try:
    with bayesian_model_nb_gamma_penalized_splines_penalty:
        ppc_posterior_nb_gamma_penalized_splines_penalty = pm.sample_posterior_predictive(idata_bayesian_nb_gamma_penalized_splines_penalty, extend_inferencedata=True, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)
try:
    with bayesian_model_nb_gamma_penalized_splines_penalty:
        ppc_prior_nb_gamma_penalized_splines_penalty = pm.sample_prior_predictive(no_ppc_samples_default, random_seed=RANDOM_SEED)
except Exception as error:
    print("Error", error)

In [ ]:
crps_nb_gamma_penalized_splines_penalty, crps_average_nb_gamma_penalized_splines_penalty = calculate_crps(idata_bayesian_nb_gamma_penalized_splines_penalty, actuals=dataY_df, posterior_predictive='obs');

In [ ]:
# Create dictionary of parameters used in the model:
divergences_psplines_penalty = count_divergences(idata_bayesian_nb_gamma_penalized_splines_penalty)
all_dictionaries = generate_all_dictionaries(
    training_data=df, 
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train, 
    data_characteristics_eval=data_characteristics_eval, 
    data_transformation=data_transformation, 
    country_name=country_name, 
    predictors=predictors, 
    model_name=model_name_penalized_splines_penalty, 
    prior_specifications=prior_specifications_penalized_splines_penalty, 
    no_spline_coefficients_per_regressor=[no_spline_coefficients_X1, no_spline_coefficients_X2],
    tau_hyperparameters=[(tauX1_ig_alpha_gaussian_sigma, tauX1_ig_beta_gaussian_sigma), (tauX2_ig_alpha_gaussian_sigma, tauX2_ig_beta_gaussian_sigma)],
    alpha_nb_hyperparameters=(gamma_alpha_nb_alpha, gamma_beta_nb_alpha),
    intercept_hyperparameters= (intercept_gaussian_mu, intercept_gaussian_sigma),
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_psplines_penalty,
    sampling_time=sampling_time_psplines_penalty, 
    idata=idata_bayesian_nb_gamma_penalized_splines_penalty,
    crps_score_train=crps_average_nb_gamma_penalized_splines_penalty,
    crps_score_eval=crps_average_nb_gamma_penalized_splines_penalty)
ml_flow_tracking(**all_dictionaries)

**7. Explore other ways to build Bayesian model with Negative Binomial distribution (alpha Gamma distributed) and penalized regression splines**

In [ ]:
# # NOTE: WIP: Implementation of penalized regression splines still to be figured out, Version with Penalty Matrix
# # Build the model with penalty matrix
# with pm.Model(coords=COORDS_splines) as bayesian_model_nb_gamma_penalized_splines_penalty_K:
#     # Specify the data
#     X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
#     X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
#     Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")
# 
# 
#     # Define the priors for regressors and  negative binomial over-dispersion parameter
#     intercept = pm.HalfNormal("intercept", sigma=0.1)
# 
#     # Define first order random walk to incorporate regularization
#     tau_X1 = pm.InverseGamma("tauX1", alpha=a_X1, beta=b_X1)
#     tau_X2 = pm.InverseGamma("tauX2", alpha=a_X2, beta=b_X2)
# 
#     # Calculate covariance matrices
#     cov_X1 = at.dot(at.nlinalg.matrix_inverse(K_X1), at.pow(tau_X1, 2))
#     cov_X2 = at.dot(at.nlinalg.matrix_inverse(K_X2), at.pow(tau_X2, 2))
#     
#     # Compute cholesky decompositions of covariance matrices
#     chol_X1 = at.slinalg.cholesky(cov_X1)
#     chol_X2 = at.slinalg.cholesky(cov_X2)
# 
#     # Define multivariate normal distributions
#     beta_X1 = pm.MvNormal("slopesX1", mu=at.zeros(no_X1_coefficients), chol=chol_X1, shape=no_X1_coefficients)
#     beta_X2 = pm.MvNormal("slopesX2", mu=at.zeros(no_X2_coefficients), chol=chol_X2, shape=no_X2_coefficients)
# 
#     alpha = pm.Gamma('alpha', 0.1, 0.1)
# 
#     # Note: Possibly an option to change to pm.Deterministic
#     # Define mean of negative binomial distribution
#     mu = pm.math.exp(intercept + pm.math.dot(at.as_tensor_variable(basis_X1), beta_X1.T) + pm.math.dot(at.as_tensor_variable(basis_X2),beta_X2.T))
# 
#     # Define the likelihood
#     obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")
# 
#     # Note: Optionally define cores and chains
#     # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
#     idata_bayesian_nb_gamma_penalized_splines_penalty_K = pm.sample(draws=1000, tune=2000, random_seed=RANDOM_SEED, target_accept=0.95)

In [ ]:
# try:
#     with bayesian_model_nb_gamma_penalized_splines_penalty_K:
#         ppc_posterior_nb_gamma_penalized_splines_penalty_K = pm.sample_posterior_predictive(idata_bayesian_nb_gamma_penalized_splines_penalty_K, extend_inferencedata=True, random_seed=RANDOM_SEED)
# except Exception as error:
#     print("Error", error)

In [ ]:
# try:
#     with bayesian_model_nb_gamma_penalized_splines_penalty_K:
#         ppc_prior_nb_gamma_penalized_splines_penalty_K = pm.sample_prior_predictive(no_ppc_samples_default, random_seed=RANDOM_SEED)
# except Exception as error:
#     print("Error", error)

In [ ]:
# # Note: Pending suggestion of ChatGPT to use tau for covariance altough it's part of the sampling process itself
# # Create a covariance matrix using Theano operations
# tau_X1_squared = tau_X1**2
# tau_X1_squared_diag = tt.diag(tau_X1_squared)
# cov_X1 = tt.dot(np.linalg.inv(K_X1), tau_X1_squared_diag)
# 
# tau_X2_squared = tau_X2**2
# tau_X2_squared_diag = tt.diag(tau_X2_squared)
# cov_X2 = tt.dot(np.linalg.inv(K_X2), tau_X2_squared_diag)

## III.2 PyStan Model Building

### Model 1: Negative Binomial distribution (alpha Gamma distributed) and penalized splines via "manual" random walk

Likelihood: negative binomial
Splines: yes
Penalization: first order random walk
Implementation: diffuse prior + manually

In [21]:
definition_stan_model_nb_gamma_penalized_splines_grw = """
data {
    int<lower=0> N;                      // Number of data points
    int<lower=1> num_basis_X1;           // Number of coefficients for P-splines for X1
    int<lower=1> num_basis_X2;           // Number of coefficients for P-splines for X2

    matrix[N, num_basis_X1] basis_X1;    // Basis matrix for regressor X1
    matrix[N, num_basis_X2] basis_X2;    // Basis matrix for regressor X2
    int Y[N];                            // Observed Target variable
    
    real a_tau_X1;                       // Shape parameter for InverseGamma of tau for X1
    real b_tau_X1;                       // Scale parameter for InverseGamma of tau for X1
    real a_tau_X2;                       // Shape parameter for InverseGamma of tau for X2
    real b_tau_X2;                       // Scale parameter for InverseGamma of tau for X2
    real a_alpha;                        // Shape parameter for Gamma of alpha
    real b_alpha;                        // Scale parameter for Gamma of alpha
    real diffuse_prior_mu;                    // Mean of spline coefficient diffuse prior
    real<lower=0> diffuse_prior_sigma;                 // Standard deviation of spline coefficient diffuse prior
    real intercept_mu;                        // Mean of intercept prior
    real<lower=1> intercept_sigma;                     // Standard deviation of intercept prior
    
    int<lower=0> N_eval;                 // Number of data points for evaluation
    matrix[N_eval, num_basis_X1] basis_X1_eval;    // Basis matrix for regressor X1_eval
    matrix[N_eval, num_basis_X2] basis_X2_eval;    // Basis matrix for regressor X2_eval
    int Y_eval[N_eval];                            // Observed Target variable for evaluation
}

parameters {
    vector[num_basis_X1] spline_coefficients_X1;
    vector[num_basis_X2] spline_coefficients_X2;
    real intercept;
    real<lower=0> tau_X1;
    real<lower=0> tau_X2;
    real<lower=0> alpha;
}

transformed parameters {
    vector[N] mu;
    mu = exp(intercept + to_vector(basis_X1 * spline_coefficients_X1) + to_vector(basis_X2 * spline_coefficients_X2));
}

model {
    intercept ~ normal(intercept_mu, intercept_sigma);
    tau_X1 ~ inv_gamma(a_tau_X1, b_tau_X1);
    tau_X2 ~ inv_gamma(a_tau_X2, b_tau_X2);

    spline_coefficients_X1[1] ~ normal(diffuse_prior_mu, diffuse_prior_sigma);      // diffuse prior for first coefficient
    for (i in 2:num_basis_X1) {                     // random walk prior for coefficients
        spline_coefficients_X1[i] ~ normal(spline_coefficients_X1[i-1], tau_X1);
    }
    
    spline_coefficients_X2[1] ~ normal(diffuse_prior_mu, diffuse_prior_sigma);      // diffuse prior for first coefficient
    for (i in 2:num_basis_X2) {                     // random walk prior for coefficients
        spline_coefficients_X2[i] ~ normal(spline_coefficients_X2[i-1], tau_X2);
    }
    
    alpha ~ gamma(0.1, 0.1);
    Y ~ neg_binomial_2(mu, alpha);
}
generated quantities {
    int y_pred_train[N];
    int y_pred_eval[N_eval];
    vector[N_eval] mu_eval;
    
    mu_eval = exp(intercept + to_vector(basis_X1_eval * spline_coefficients_X1) + to_vector(basis_X2_eval * spline_coefficients_X2));
    for (n in 1:N) {
        y_pred_train[n] = neg_binomial_2_rng(mu[n], alpha);
    }
    for (n_eval in 1:N_eval) {
        y_pred_eval[n_eval] = neg_binomial_2_rng(mu_eval[n_eval], alpha);
    }
}
"""

In [22]:
# Compile the model
stan_model_nb_gamma_penalized_splines_grw = pystan.StanModel(model_code=definition_stan_model_nb_gamma_penalized_splines_grw)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_230c9f40206178d84ce58224164f070c NOW.


In [23]:
stan_data_nb_gamma_penalized_splines_grw = {
    "N": len(dataY),
    "num_basis_X1": no_spline_coefficients_X1,
    "num_basis_X2": no_spline_coefficients_X2,
    "basis_X1": np.asarray(basis_X1, order="F"),
    "basis_X2": np.asarray(basis_X2, order="F"),
    "Y": dataY.astype(int),
    "a_tau_X1": tauX1_ig_alpha_gaussian_sigma,
    "b_tau_X1": tauX1_ig_beta_gaussian_sigma,
    "a_tau_X2": tauX2_ig_alpha_gaussian_sigma,
    "b_tau_X2": tauX2_ig_beta_gaussian_sigma,
    "a_alpha": gamma_alpha_nb_alpha,
    "b_alpha": gamma_beta_nb_alpha,
    "diffuse_prior_mu": diffuse_prior_beta_mu,
    "diffuse_prior_sigma": diffuse_prior_beta_sigma,
    "intercept_mu": intercept_gaussian_mu,
    "intercept_sigma": intercept_gaussian_sigma,
    "N_eval": len(dataY_eval),
    "basis_X1_eval": np.asarray(basis_X1_eval, order="F"),
    "basis_X2_eval": np.asarray(basis_X2_eval, order="F"),
    "Y_eval": dataY_eval.astype(int)
}

In [24]:
# Sampling from the model
start_time = time.time()
stan_model_nb_gamma_penalized_splines_grw_sampling = stan_model_nb_gamma_penalized_splines_grw.sampling(data=stan_data_nb_gamma_penalized_splines_grw, iter=draws_default, chains=no_chains_default, seed=RANDOM_SEED, control={"adapt_delta": target_accept_default})
sampling_time_nb_gamma_penalized_splines_grw_stan = time.time() - start_time

In [25]:
idata_bayesian_nb_gamma_penalized_splines_grw_stan = az.from_pystan(
    posterior=stan_model_nb_gamma_penalized_splines_grw_sampling,
    posterior_predictive=["y_pred_train", "y_pred_eval"])

In [26]:
# Note: possibility to export idata as example object
# idata_bayesian_nb_gamma_penalized_splines_grw_stan.to_netcdf("temp_data/idata_bayesian_nb_gamma_penalized_splines_grw_stan.nc")

'temp_data/idata_bayesian_nb_gamma_penalized_splines_grw_stan.nc'

In [ ]:
# Note: possibility to import idata as example object
# idata_bayesian_nb_gamma_penalized_splines_grw_stan = az.from_netcdf("temp_data/idata_bayesian_nb_gamma_penalized_splines_grw_stan.nc")

In [27]:
crps_train_nb_gamma_penalized_splines_grw_stan, crps_train_average_nb_gamma_penalized_splines_grw_stan = calculate_crps(idata_bayesian_nb_gamma_penalized_splines_grw_stan, dataY_df, posterior_predictive="y_pred_train")
crps_eval_nb_gamma_penalized_splines_grw_stan, crps_eval_average_nb_gamma_penalized_splines_grw_stan = calculate_crps(idata_bayesian_nb_gamma_penalized_splines_grw_stan, dataY_eval_df, posterior_predictive="y_pred_eval")

C:\Users\Uwe Drauz\anaconda3\envs\bachelor_thesis_pystan_clean\lib\site-packages\CRPS\CRPS.py:147: RuntimeWarning: overflow encountered in long_scalars
  self.__delta_fc = np.array([all_mem[n+1] - all_mem[n] for n in range(len(all_mem)-1)] + list(np.zeros(1)), dtype=object)
C:\Users\Uwe Drauz\anaconda3\envs\bachelor_thesis_pystan_clean\lib\site-packages\CRPS\CRPS.py:141: RuntimeWarning: overflow encountered in long_scalars
  self.__delta_fc = np.array([all_mem[n+1] - all_mem[n] for n in range(len(all_mem)-1)], dtype=object)
C:\Users\Uwe Drauz\anaconda3\envs\bachelor_thesis_pystan_clean\lib\site-packages\CRPS\CRPS.py:157: RuntimeWarning: overflow encountered in long_scalars
  delta_fc.append(self.fc[f+1] - self.fc[f])


In [ ]:
# Create dictionary of parameters used in the model:
divergences_splines_stan = count_divergences(idata_bayesian_nb_gamma_penalized_splines_grw_stan)
all_dictionaries_penalized_splines_grw = generate_all_dictionaries(
    training_data=df,
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train, 
    data_characteristics_eval=data_characteristics_eval,
    data_transformation=data_transformation, 
    country_name=country_name, 
    predictors=predictors, 
    model_name=model_name_penalized_splines_grw, 
    prior_specifications=prior_specifications_penalized_splines_grw,
    alpha_nb_hyperparameters=(gamma_alpha_nb_alpha, gamma_beta_nb_alpha),
    intercept_hyperparameters= (intercept_gaussian_mu, intercept_gaussian_sigma),
    no_spline_coefficients_per_regressor=[no_spline_coefficients_X1, no_spline_coefficients_X2],
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_splines_stan,
    sampling_time=sampling_time_nb_gamma_penalized_splines_grw_stan, 
    idata=idata_bayesian_nb_gamma_penalized_splines_grw_stan,
    crps_score_train=crps_train_average_nb_gamma_penalized_splines_grw_stan,
    crps_score_eval=crps_eval_average_nb_gamma_penalized_splines_grw_stan)
ml_flow_tracking(**all_dictionaries_penalized_splines_grw)

### Model 2: Negative Binomial distribution (alpha Gamma distributed) and penalized splines via multivariate distribution using penalty matrix

In [34]:
definition_stan_model_nb_gamma_penalized_splines_multivariate = """
data {
    int<lower=1> N;          // Number of data points
    int<lower=1> num_basis_X1;  // Number of coefficients for P-splines for X1
    int<lower=1> num_basis_X2;  // Number of coefficients for P-splines for X2

    matrix[N, num_basis_X1] basis_X1;  // Basis matrix for regressor X1
    matrix[N, num_basis_X2] basis_X2;  // Basis matrix for regressor X2
    int Y[N];             // Observed Target variable
    
    matrix[num_basis_X1, num_basis_X1] K_X1;  // Penalty matrix for X1
    matrix[num_basis_X2, num_basis_X2] K_X2;  // Penalty matrix for X2
    
    real a_tau_X1;              // Shape parameter for InverseGamma of tau for X1
    real b_tau_X1;              // Scale parameter for InverseGamma of tau for X1
    real a_tau_X2;            // Shape parameter for InverseGamma of tau for X2
    real b_tau_X2;            // Scale parameter for InverseGamma of tau for X2
    real a_alpha;             // Shape parameter for Gamma of alpha
    real b_alpha;             // Scale parameter for Gamma of alpha
    real intercept_mu;        // Mean of intercept prior
    real<lower=0> intercept_sigma;     // Standard deviation of intercept prior
    
    int<lower=0> N_eval;                 // Number of data points for evaluation
    matrix[N_eval, num_basis_X1] basis_X1_eval;    // Basis matrix for regressor X1_eval
    matrix[N_eval, num_basis_X2] basis_X2_eval;    // Basis matrix for regressor X2_eval
    int Y_eval[N_eval];                            // Observed Target variable for evaluation
}

parameters {
    real intercept;                           // regression intercept
    vector[num_basis_X1] spline_coefficients_X1; // Spline coefficients for X1
    vector[num_basis_X2] spline_coefficients_X2; // Spline coefficients for X2
    real<lower=0> tau_X1;                        // Precision parameter for Gaussian random walk for X1
    real<lower=0> tau_X2;                      // Precision parameter for Gaussian random walk for X2
    real<lower=0> alpha;                      // Negative Binomial dispersion parameter
}
transformed parameters {
    vector[N] mu; // Mean of Negative Binomial distribution
    matrix[num_basis_X1, num_basis_X1] precision_matrix_X1;  // precision matrix for X1
    matrix[num_basis_X2, num_basis_X2] precision_matrix_X2;  // precision matrix for X2

    mu = exp(intercept + basis_X1 * spline_coefficients_X1 + basis_X2 * spline_coefficients_X2); //taking exponential as a link function of a GAM

    precision_matrix_X1 = K_X1 / pow(tau_X1, 2);  // Defining precision matrix for X1
    precision_matrix_X2 = K_X2 / pow(tau_X2, 2);  // Defining precision matrix for X2
}

model {
    // Priors
    intercept ~ normal(0, 10);               // Diffuse prior for intercept
    tau_X1 ~ inv_gamma(a_tau_X1, b_tau_X1);  // Inverse Gamma prior for tau_X1
    tau_X2 ~ inv_gamma(a_tau_X2, b_tau_X2);  // Inverse Gamma prior for tau_X2
    alpha ~ gamma(0.1, 0.1);              // Gamma prior for alpha
    
    spline_coefficients_X1 ~ multi_normal_prec(rep_vector(0, num_basis_X1), precision_matrix_X1);  // Prior for spline coefficients for X1
    spline_coefficients_X2 ~ multi_normal_prec(rep_vector(0, num_basis_X2), precision_matrix_X2);  // Prior for spline coefficients for X2

    // Likelihood
    Y ~ neg_binomial_2(mu, alpha);
}
generated quantities{
    int y_pred_train[N];
    int y_pred_eval[N_eval];
    vector[N_eval] mu_eval;
    
    mu_eval = exp(intercept + basis_X1_eval * spline_coefficients_X1 + basis_X2_eval * spline_coefficients_X2);
    for (n in 1:N) {
        y_pred_train[n] = neg_binomial_2_rng(mu[n], alpha);
    }
    for (n_eval in 1:N_eval) {
        y_pred_eval[n_eval] = neg_binomial_2_rng(mu_eval[n_eval], alpha);
    }
}   
"""



In [35]:
# Compile the model
stan_model_nb_gamma_penalized_splines_multivariate = pystan.StanModel(model_code=definition_stan_model_nb_gamma_penalized_splines_multivariate)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7e45e14c9e53ce0eca39a351c9e98e63 NOW.


In [36]:
stan_data_nb_gamma_penalized_splines_multivariate = {
    "N": len(dataY),
    "num_basis_X1": no_spline_coefficients_X1,
    "num_basis_X2": no_spline_coefficients_X2,
    "basis_X1": np.asarray(basis_X1, order="F"),
    "basis_X2": np.asarray(basis_X2, order="F"),
    "Y": dataY.astype(int),
    "K_X1": K_X1,
    "K_X2": K_X2,
    "a_tau_X1": tauX1_ig_alpha_gaussian_sigma,
    "b_tau_X1": tauX1_ig_beta_gaussian_sigma,
    "a_tau_X2": tauX2_ig_alpha_gaussian_sigma,
    "b_tau_X2": tauX2_ig_beta_gaussian_sigma,
    "a_alpha": gamma_alpha_nb_alpha,
    "b_alpha": gamma_beta_nb_alpha,
    "intercept_mu": intercept_gaussian_mu,
    "intercept_sigma": intercept_gaussian_sigma,
    "N_eval": len(dataY_eval),
    "basis_X1_eval": np.asarray(basis_X1_eval, order="F"),
    "basis_X2_eval": np.asarray(basis_X2_eval, order="F"),
    "Y_eval": dataY_eval.astype(int)
}

In [37]:
# Sampling from the model
start_time = time.time()
stan_model_nb_gamma_penalized_splines_multivariate_sampling = stan_model_nb_gamma_penalized_splines_multivariate.sampling(data=stan_data_nb_gamma_penalized_splines_multivariate, iter=draws_default, chains=no_chains_default, seed=RANDOM_SEED, control={"adapt_delta": target_accept_default})
sampling_time_nb_gamma_penalized_splines_multivariate_stan = time.time() - start_time

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [38]:
idata_bayesian_nb_gamma_penalized_splines_multivariate_stan = az.from_pystan(
    posterior=stan_model_nb_gamma_penalized_splines_multivariate_sampling,
    posterior_predictive=["y_pred_train", "y_pred_eval"])

In [40]:
# Note: possibility to export idata as example object
# idata_bayesian_nb_gamma_penalized_splines_multivariate_stan.to_netcdf("temp_data/idata_bayesian_nb_gamma_penalized_splines_multivariate_stan.nc")

'temp_data/idata_bayesian_nb_gamma_penalized_splines_multivariate_stan.nc'

In [ ]:
# Note: possibility to import idata as example object
# idata_bayesian_nb_gamma_penalized_splines_multivariate_stan = az.from_netcdf("temp_data/idata_bayesian_nb_gamma_penalized_splines_multivariate_stan.nc")

In [39]:
crps_train_nb_gamma_penalized_splines_multivariate_stan, crps_train_average_nb_gamma_penalized_splines_multivariate_stan = calculate_crps(idata_bayesian_nb_gamma_penalized_splines_multivariate_stan, dataY_df, posterior_predictive="y_pred_train")
crps_eval_nb_gamma_penalized_splines_multivariate_stan, crps_eval_average_nb_gamma_penalized_splines_multivariate_stan = calculate_crps(idata_bayesian_nb_gamma_penalized_splines_multivariate_stan, dataY_eval_df, posterior_predictive="y_pred_eval")

In [ ]:
# Create dictionary of parameters used in the model:
divergences_psplines_penalty_stan = count_divergences(idata_bayesian_nb_gamma_penalized_splines_penalty)
all_dictionaries = generate_all_dictionaries(
    training_data=df,
    evaluation_data=df_eval,
    data_characteristics_train=data_characteristics_train,
    data_characteristics_eval=data_characteristics_eval,
    data_transformation=data_transformation,
    country_name=country_name,
    predictors=predictors,
    model_name=model_name_penalized_splines_penalty,
    prior_specifications=prior_specifications_penalized_splines_penalty,
    no_spline_coefficients_per_regressor=[no_spline_coefficients_X1, no_spline_coefficients_X2],
    tau_hyperparameters=[(tauX1_ig_alpha_gaussian_sigma, tauX1_ig_beta_gaussian_sigma),
                         (tauX2_ig_alpha_gaussian_sigma, tauX2_ig_beta_gaussian_sigma)],
    alpha_nb_hyperparameters=(gamma_alpha_nb_alpha, gamma_beta_nb_alpha),
    intercept_hyperparameters=(intercept_gaussian_mu, intercept_gaussian_sigma),
    tuning_iterations=tune_default,
    sampling_iterations=draws_default,
    target_acceptance_rate=target_accept_default,
    chains=no_chains_default,
    divergences=divergences_psplines_penalty,
    sampling_time=sampling_time_psplines_penalty,
    idata=idata_bayesian_nb_gamma_penalized_splines_penalty,
    crps_score_train=crps_average_nb_gamma_penalized_splines_penalty,
    crps_score_eval=crps_average_nb_gamma_penalized_splines_penalty)
ml_flow_tracking(**all_dictionaries)

## IV.1 PyMc Model Evaluation

### 1.Model with Gaussian distribution for likelihood

In [ ]:
# Specify which model should be examined
model = bayesian_model_gauss
idata = idata_bayesian_gauss

**Visualisations**

In [ ]:
az.plot_trace(idata, figsize=(10, 9))
az.plot_trace(idata, combined=True, figsize=(10, 9))
az.plot_posterior(idata, figsize=(10, 3))

In [ ]:
az.plot_ppc(idata, num_pp_samples=100, kind="scatter")

**Summary Statistics**

In [ ]:
az.summary(idata.posterior, var_names=["intercept", "slopes"])

In [ ]:
az.summary(idata.posterior, var_names=["sigma"])

**CRPS evaluation**

### 2.Model with negative binomial distribution for likelihood (alpha half normal distributed)

In [ ]:
# Specify which model should be examined
model = bayesian_model_nb_half_normal
idata = idata_bayesian_nb_half_normal

**Visualisations**

In [ ]:
# Plotting trace with parameters transformed back to original scale
# Plotting trace plots with appropriate size
az.plot_trace(np.exp(idata.posterior), var_names=["intercept", "slopes"], combined=True, figsize=(10, 6))
az.plot_trace(idata.posterior, var_names=["alpha"], combined=True, figsize=(10, 3))

# Plotting posterior plots with appropriate size
az.plot_posterior(np.exp(idata.posterior), var_names=["intercept", "slopes"], figsize=(12, 3))
az.plot_posterior(idata, var_names=["alpha"], figsize=(4, 3))


In [ ]:
# Plotting trace plots in logaritmic scale
az.plot_trace(idata, figsize=(10, 9))
az.plot_trace(idata, combined=True, figsize=(10, 9))
az.plot_posterior(idata, figsize=(10, 3))

In [ ]:
try:
    az.plot_ppc(idata, num_pp_samples=50, kind="scatter")
except Exception as error:
    print("Error", error)

**Summary Statistics**

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior), var_names=["intercept", "slopes"])

In [ ]:
az.summary(idata.posterior, var_names=["alpha"])

**CRPS evaluation**

### 3. Model with negative binomial distribution for likelihood (alpha Gamma distributed)

In [ ]:
# Specify which model should be examined
model = bayesian_model_nb_gamma
idata = idata_bayesian_nb_gamma

**Visualisations**

In [ ]:
# Plotting trace with parameters transformed back to original scale
# Plotting trace plots with appropriate size
az.plot_trace(np.exp(idata.posterior), var_names=["intercept", "slopes"], combined=True, figsize=(10, 6))
az.plot_trace(idata.posterior, var_names=["alpha"], combined=True, figsize=(10, 3))

In [ ]:
# Plotting posterior plots with appropriate size
az.plot_posterior(np.exp(idata.posterior), var_names=["intercept", "slopes"], figsize=(12, 3))
az.plot_posterior(idata, var_names=["alpha"], figsize=(4, 3))

In [ ]:
# Plotting trace plots in logaritmic scale
az.plot_trace(idata, figsize=(10, 9))
az.plot_trace(idata, combined=True, figsize=(10, 9))
az.plot_posterior(idata, figsize=(10, 3))

In [ ]:
try:
    az.plot_ppc(idata, num_pp_samples=50, kind="scatter")
except Exception as error:
    print("Error", error)

**Summary Statistics**

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior), var_names=["intercept", "slopes"])

In [ ]:
az.summary(idata.posterior, var_names=["alpha"])

**CRPS evaluation**

### 4. Model with Negative Binomial distribution (alpha Gamma distributed) and regression splines (B-splines)

In [ ]:
# Specify which model should be examined
model = bayesian_model_nb_gamma_splines
idata = idata_bayesian_nb_gamma_splines

**Visualisations**

In [ ]:
# Plotting trace with parameters transformed back to original scale
# Plotting trace plots with appropriate size
az.plot_trace(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"], combined=True, figsize=(10, 6))
az.plot_trace(idata.posterior, var_names=["alpha"], combined=True, figsize=(10, 3))

In [ ]:
# Plotting posterior plots with appropriate size
az.plot_posterior(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])
az.plot_posterior(idata, var_names=["alpha"], figsize=(4, 3))

In [ ]:
# Plotting trace plots in logaritmic scale
az.plot_trace(idata, figsize=(10, 9))
az.plot_trace(idata, combined=True, figsize=(10, 9))
az.plot_posterior(idata)

In [ ]:
try:
    az.plot_ppc(idata, num_pp_samples=50, kind="scatter")
except Exception as error:
    print("Error", error)

**Summary Statistics**

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])

In [ ]:
az.summary(idata.posterior, var_names=["alpha"])

**CRPS evaluation**

### 5. Model with Negative Binomial distribution (alpha Gamma distributed) and penalized regression splines via GRW

In [ ]:
# Specify which model should be examined
model = bayesian_model_nb_gamma_penalized_splines_grw
idata = idata_bayesian_nb_gamma_penalized_splines_grw

**Summary Statistics**

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])

In [ ]:
az.summary(idata.posterior, var_names=["alpha"])

**Visualisations**

In [ ]:
# Plotting trace with parameters transformed back to original scale
# Plotting trace plots with appropriate size
az.plot_trace(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"], combined=True, figsize=(10, 6))
az.plot_trace(idata.posterior, var_names=["alpha"], combined=True, figsize=(10, 3))

In [ ]:
# Plotting posterior plots with appropriate size
az.plot_posterior(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])
az.plot_posterior(idata, var_names=["alpha"], figsize=(4, 3))

In [ ]:
az.plot_posterior(idata.posterior, var_names=["slopesX1"], figsize=(25, 18))

In [ ]:
# Plotting trace plots in logaritmic scale
az.plot_trace(idata)
az.plot_trace(idata, combined=True)
az.plot_posterior(idata)

In [ ]:
try:
    az.plot_ppc(idata, num_pp_samples=50, kind="scatter")
except Exception as error:
    print("Error", error)

### 6. Model with Negative Binomial distribution (alpha Gamma distributed) and penalized regression splines via pymc Potential

In [ ]:
# Specify which model should be examined
model = bayesian_model_nb_gamma_penalized_splines_penalty
idata = idata_bayesian_nb_gamma_penalized_splines_penalty

**Summary Statistics**

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])

In [ ]:
az.summary(idata.posterior, var_names=["alpha"])

**Visualisations**

In [ ]:
# Plotting trace with parameters transformed back to original scale
# Plotting trace plots with appropriate size
az.plot_trace(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"], combined=True, figsize=(10, 6))
az.plot_trace(idata.posterior, var_names=["alpha"], combined=True, figsize=(10, 3))

In [ ]:
# Plotting posterior plots with appropriate size
az.plot_posterior(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])
az.plot_posterior(idata, var_names=["alpha"], figsize=(4, 3))

In [ ]:
az.plot_posterior(idata.posterior, var_names=["slopesX1"], figsize=(25, 18))

In [ ]:
# Plotting trace plots in logaritmic scale
az.plot_trace(idata)
az.plot_trace(idata, combined=True)
az.plot_posterior(idata)

In [ ]:
try:
    az.plot_ppc(idata, num_pp_samples=50, kind="scatter")
except Exception as error:
    print("Error", error)

**CRPS evaluation**

In [ ]:
print(crps_average_gauss)
print(crps_average_nb_half_normal)
print(crps_average_nb_gamma)
print(crps_average_nb_gamma_splines)
print(crps_average_nb_gamma_penalized_splines_grw)
print(crps_average_nb_gamma_penalized_splines_penalty)

## IV.2 PyStan Model Evaluation

### 1. Model with Negative Binomial distribution (alpha Gamma distributed) and penalized regression splines via GRW

In [ ]:
# Specify which model should be examined
model = stan_model_nb_gamma_penalized_splines_grw
idata = idata_bayesian_nb_gamma_penalized_splines_grw_stan

**Summary Statistics**

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior))

In [ ]:
az.summary(idata.posterior)

**Visualisations**

In [ ]:
# Plotting trace with parameters transformed back to original scale
# Plotting trace plots with appropriate size
az.plot_trace(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"], combined=True, figsize=(10, 6))
az.plot_trace(idata.posterior, var_names=["alpha"], combined=True, figsize=(10, 3))

In [ ]:
# Plotting posterior plots with appropriate size
az.plot_posterior(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])
az.plot_posterior(idata, var_names=["alpha"], figsize=(4, 3))

In [ ]:
az.plot_posterior(idata.posterior, var_names=["slopesX1"], figsize=(25, 18))

In [ ]:
# Plotting trace plots in logaritmic scale
az.plot_trace(idata)
az.plot_trace(idata, combined=True)
az.plot_posterior(idata)

In [ ]:
try:
    az.plot_ppc(idata, num_pp_samples=50, kind="scatter")
except Exception as error:
    print("Error", error)

### 2. Model with Negative Binomial distribution (alpha Gamma distributed) and penalized regression splines via multivariate distribution using penalty matrix

In [ ]:
# Specify which model should be examined
model = stan_model_nb_gamma_penalized_splines_multivariate
idata = idata_bayesian_nb_gamma_penalized_splines_multivariate_stan

**Summary Statistics**

In [ ]:
# Transform coefficients to recover parameter values
az.summary(np.exp(idata.posterior))

In [ ]:
az.summary(idata.posterior)

**Visualisations**

In [ ]:
# Plotting trace with parameters transformed back to original scale
# Plotting trace plots with appropriate size
az.plot_trace(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"], combined=True, figsize=(10, 6))
az.plot_trace(idata.posterior, var_names=["alpha"], combined=True, figsize=(10, 3))

In [ ]:
# Plotting posterior plots with appropriate size
az.plot_posterior(np.exp(idata.posterior), var_names=["intercept", "slopesX1", "slopesX2"])
az.plot_posterior(idata, var_names=["alpha"], figsize=(4, 3))

In [ ]:
az.plot_posterior(idata.posterior, var_names=["slopesX1"], figsize=(25, 18))

In [ ]:
# Plotting trace plots in logaritmic scale
az.plot_trace(idata)
az.plot_trace(idata, combined=True)
az.plot_posterior(idata)

In [ ]:
try:
    az.plot_ppc(idata, num_pp_samples=50, kind="scatter")
except Exception as error:
    print("Error", error)

## V. Test functionalities with simulated data

## VI. Data exploration steps

In [ ]:
idata = idata_bayesian_nb_gamma_splines
az.ess(idata, var_names=["intercept", "slopesX1", "slopesX2"])

In [ ]:
idata = idata_bayesian_nb_gamma_penalized_splines_penalty
az.ess(idata, var_names=["intercept", "slopesX1", "slopesX2"])

In [ ]:
pm.math.exp(pm.math.exp(2))

In [ ]:
np.exp(np.exp(2))

In [ ]:
data_cm_features_allyears.loc[data_cm_features_allyears["country_id"] == country_id, 'ged_sb']

In [ ]:
az.summary(idata)

In [ ]:
posterior_predictive = ppc_posterior_nb_gamma.get("posterior_predicitve"); posterior_predictive

In [ ]:
ppc_posterior_nb_gamma

In [ ]:
posterior_predictive["obs"]

In [ ]:
az.plot_bpv(idata, figsize=(12, 6))

In [ ]:
posterior_samples =  idata["posterior_predictive"]["obs"].values
 # Reshape the array to a 2D array with samples in rows and observations in columns
num_chains, num_draws, num_observations = posterior_samples.shape
reshaped_samples = posterior_samples.reshape(num_chains * num_draws, num_observations)

# Convert the 2D array to a pandas DataFrame
df_posterior_samples = pd.DataFrame(reshaped_samples)

# Optionally, you can add meaningful column names to the DataFrame
# For example, assuming the original DataFrame has column names 'obs_0', 'obs_1', etc.
column_names = ['obs_{}'.format(column_idx) for column_idx in range(num_observations)]
df_posterior_samples.columns = column_names

### Test model with simulated data

In [ ]:
##Step 1: Import Libraries
import pymc as pm
import pandas as pd
import patsy as ps
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import BSpline

##Step 2: Generate simulated data
# Set random seed for reproducibility
RANDOM_SEED = 12345

# Number of samples
n_samples = 500

# Generate X values
X1 = np.sort(np.random.rand(n_samples) * 10)
X2 = np.sort(np.random.rand(n_samples) * 10)

# Number of knots
num_knots = 10

# Generate spline curves for X1 and X2
knots1 = np.linspace(0, 10, num_knots)
knots2 = np.linspace(0, 10, num_knots)

coeffs1 = np.random.randn(num_knots)
coeffs2 = np.random.randn(num_knots)

spline1 = BSpline(knots1, coeffs1, k=3)
spline2 = BSpline(knots2, coeffs2, k=3)

f1_values = spline1(X1)
f2_values = spline2(X2)

# Generate Y values
beta_0 = 0.5
beta_1 = 1
beta_2 = 1.2

mu = np.exp(beta_0 + beta_1 * f1_values + beta_2 * f2_values)
n = 2.0  # Adjust the size parameter as needed
p = mu / (mu + n)  # Adjust the probability parameter as needed
Y = np.random.negative_binomial(n, p, size=n_samples)

# Create a DataFrame
df = pd.DataFrame({
    "ged_sb_tlag_1": X1,
    "ged_sb_tsum_24": X2,
    "ged_sb": Y
})

## Step 3: Visualize the Simulated Data
# Visualizing data
plt.figure(figsize=(15, 5))

# Plot X1 vs. Y
plt.subplot(1, 2, 1)
plt.scatter(df['ged_sb_tlag_1'], df['ged_sb'], color='blue', s=10, alpha=0.5)
plt.plot(X1, mu, color='red', linewidth=2)
plt.title('X1 vs. Y')
plt.xlabel('ged_sb_tlag_1')
plt.ylabel('ged_sb')

# Plot X2 vs. Y
plt.subplot(1, 2, 2)
plt.scatter(df['ged_sb_tsum_24'], df['ged_sb'], color='green', s=10, alpha=0.5)
plt.plot(X2, mu, color='red', linewidth=2)
plt.title('X2 vs. Y')
plt.xlabel('ged_sb_tsum_24')
plt.ylabel('ged_sb')

plt.tight_layout()
plt.show()


##Step 4: Define the PyMc Model
dataX1 = df.ged_sb_tlag_1.to_numpy()
dataX2 = df.ged_sb_tsum_24.to_numpy()
dataY = df.ged_sb.to_numpy()

# Note: If knots are intended to be not equally spaces (e.g. set by quantiles) the consequence on penalized splines has to be evaluated
# Define the knot positions


# We define the knot list such that we'll have num_knots knot inbetween the boundaries of the data
knot_list_X1 = np.linspace(dataX1.min(), dataX1.max(), num_knots+2)[1:-1]
knot_list_X2 = np.linspace(dataX2.min(), dataX2.max(), num_knots+2)[1:-1]

# Define the basis functions
# Note: We include the intercept once in the model via the intercept parameter, to prevent having additional '1' columns in basis functions include_intercept is set to False and automatic creation of '1' column of patsy is prevented by '-1'
basis_X1 = ps.dmatrix("bs(X1, knots=knots, degree=3, include_intercept=False) -1", {"X1": dataX1, "knots": knot_list_X1})
basis_X2 = ps.dmatrix("bs(X2, knots=knots, degree=3, include_intercept=False) -1", {"X2": dataX2, "knots": knot_list_X2})

# Define variables for tau priors
a_X1 = a_X2 = 1
b_X1 = b_X2 = 0.005

def generate_penalty_matrix(no_coefficients):
    penalty_matrix = -2 * np.eye(no_coefficients) + np.eye(no_coefficients, k=1) + np.eye(no_coefficients, k=-1)
    penalty_matrix[0, 0] = 1
    penalty_matrix[no_coefficients - 1, no_coefficients - 1] = 1
    return penalty_matrix


no_X1_coefficients = basis_X1.shape[1]
no_X2_coefficients = basis_X2.shape[1]
K_X1 = generate_penalty_matrix(no_X1_coefficients)
K_X2 = generate_penalty_matrix(no_X2_coefficients)

COORDS_penalized_splines = \
    {"obs_idx": df.index, "splines_x1_rest": np.arange(basis_X1.shape[1] -1), "splines_x2_rest": np.arange(basis_X2.shape[1] -1)}

# NOTE: WIP: Implementation of penalized regression splines still to be figured out, Version with Penalty deduction in likelihood
with pm.Model(coords=COORDS_penalized_splines) as bayesian_model_nb_gamma_penalized_splines_penalty:
    # Specify the data
    X1 = pm.ConstantData("fatality lag t=1", dataX1, dims="obs_idx")
    X2 = pm.ConstantData("fatality rolling average t=24", dataX2, dims="obs_idx")
    Y = pm.ConstantData("fatility count", dataY, dims="obs_idx")

    # Define the priors for regressors and negative binomial over-dispersion parameter
    intercept = pm.Normal("intercept", sigma=10)

    # Define first order random walk to incorporate regularization
    tau_X1 = pm.InverseGamma("tauX1", alpha=a_X1, beta=b_X1)
    tau_X2 = pm.InverseGamma("tauX2", alpha=a_X2, beta=b_X2)

    # Create deterministic variables for squared taus
    tau_X1_squared = pm.Deterministic("tau_X1_squared", tau_X1 ** 2)
    tau_X2_squared = pm.Deterministic("tau_X2_squared", tau_X2 ** 2)

    # Introduce flat priors for the first regression coeffcients of the splines
    beta_X1_0 = pm.Normal(0, 1000, "slopesX1_0")
    beta_X2_0 = pm.Normal(0, 1000, "slopesX2_0")
    
    # Define remaining betas as normal variables with standard deviation tau
    beta_X1_rest = pm.Normal("slopesX1_rest", 0.0, tau_X1, dims="splines_x1_rest")
    beta_X2_rest = pm.Normal("slopesX2_rest", 0.0, tau_X2, dims="splines_x2_rest")
    
    # Concatenate the flat priors to the beginning of the coefficients
    beta_X1 = pm.Deterministic("slopesX1", pm.math.concatenate([[beta_X1_0], beta_X1_rest]))
    beta_X2 = pm.Deterministic("slopesX2", pm.math.concatenate([[beta_X2_0], beta_X2_rest]))


    alpha = pm.Gamma('alpha', 0.1, 0.1)

    # Define mean of negative binomial distribution
    mu = pm.math.exp(intercept + pm.math.dot(np.asarray(basis_X1, order="F"), beta_X1.T) + pm.math.dot(np.asarray(basis_X2, order="F"), beta_X2.T))

    # Define the likelihood
    obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

    # Define the penalty term and incorporate it using pm.Potential
    penalty_term_X1 = pm.math.dot(beta_X1.T, pm.math.dot(K_X1, beta_X1)) / (2 * tau_X1_squared)
    penalty_X1 = pm.Potential("penalty_X1", -penalty_term_X1)

    penalty_term_X2 = pm.math.dot(beta_X2, pm.math.dot(K_X2, beta_X2)) / (2 * tau_X2_squared)
    penalty_X2 = pm.Potential("penalty_X2", -penalty_term_X2)

    # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
    idata_bayesian_nb_gamma_penalized_splines_penalty = pm.sample(draws=draws_default, tune=tune_default, random_seed=RANDOM_SEED, target_accept=target_accept_default)

### Code which is WIP

In [ ]:
# Note: WIP
def model_training(df:pd.DataFrame, model_params: dict, validation_year: int, K:int):
    # Extracting year and month from month_id
    df['year'] = ((df['month_id'] - 1) // 12) + 1990
    df['month'] = ((df['month_id'] - 1) % 12) + 1

    # Step 1: Split data into training and test sets
    train_data = df[(df['year'] >= 1990) & (df['year'] < 2018)]
    test_data = df[df['year'] == 2018]

    # Step 2: Perform "10-Fold" time series cross-validation within the training set with a 2-month-gap
    tscv = TimeSeriesSplit(n_splits=K)

    for train_index, val_index in tscv.split(train_data):
        # Extract train and validation sets for the current fold
        train_fold = train_data.iloc[train_index]
        val_fold = train_data.iloc[val_index]

        # Exclude last two months (November and December) from the training set
        train_fold = train_fold[~((train_fold['month'] == 11) | (train_fold['month'] == 12))]

        # Step 3: Train the model on the training set
        model, trace = build_bayesian_model(df, model_params)

        # Step 4: Predict the validation set
        crps_scores, crps_average = calculate_crps(trace, val_fold)


# Note WIP
model_params = {
    "coords": COORDS,
    "intercept_prior_sigma": 0.1,
    "slopes_prior_mu": 0.0,
    "slopes_prior_sd": 10,
    "alpha_prior_alpha": 0.1,
    "alpha_prior_beta": 0.1,
    "regressors_data": [df.ged_sb_tlag_1.to_numpy(), df.ged_sb_tsum_24.to_numpy()],  # Add more predictors' data as needed
    "Y_data": df.ged_sb.to_numpy(),
    "regularization_param": 0.01,  # Add your desired regularization parameter value
    "num_knots": 10,  # Add your desired number of knots for P-splines
    "draws": 1000,
    "tune": 2000,
    "random_seed": rng,
    "target_accept": 0.95
}

# Note: WIP
def build_bayesian_model(df: pd.DataFrame(), model_params: dict):
    """
    Build the Bayesian model for the negative binomial regression with P-splines.

    Parameters:
        df (pd.DataFrame): DataFrame containing the data.
        model_params (dict): Dictionary containing the model parameters.

    Returns:
        model: PyMC3 model object
        trace: PyMC3 trace object
    """
    # Extract the model parameters
    coords = model_params["coords"]
    intercept_prior_sigma = model_params["intercept_prior_sigma"]
    slopes_prior_mu = model_params["slopes_prior_mu"]
    slopes_prior_sd = model_params["slopes_prior_sd"]
    alpha_prior_alpha = model_params["alpha_prior_alpha"]
    alpha_prior_beta = model_params["alpha_prior_beta"]
    regressors_data = model_params["regressors_data"]
    Y_data = model_params["Y_data"]
    regularization_param = model_params["regularization_param"]
    num_knots = model_params["num_knots"]
    draws = model_params["draws"]
    tune = model_params["tune"]
    random_seed = model_params["random_seed"]
    target_accept = model_params["target_accept"]

    # Build the model
    with pm.Model(coords=coords) as bayesian_model_nb:
        # Define the priors for the intercept, slopes, and over-dispersion parameter
        intercept = pm.Normal("intercept", 0.0, intercept_prior_sigma)
        slopes = pm.Normal("slopes", slopes_prior_mu, slopes_prior_sd, dims="predictors")
        alpha = pm.Gamma('alpha', alpha_prior_alpha, alpha_prior_beta)

        # Specify the data
        X = []
        for i in range(len(regressors_data)):
            X.append(pm.ConstantData("X{}".format(i), regressors_data[i], dims="obs_idx"))
        Y = pm.ConstantData("Y", Y_data, dims="obs_idx")

        # Define the basis functions
        knots = np.linspace(df['month_id'].min(), df['month_id'].max(), num_knots + 2)[1:-1]
        basis_funcs = pm.spline.make_spline_basis(knots, degree=3)

        # Define the coefficients for the basis functions
        beta = pm.Normal("beta", mu=0, sigma=regularization_param, dims="basis_funcs")

        # Define the mean of the negative binomial distribution
        mu = pm.math.exp(intercept + pm.math.dot(X, slopes) + pm.math.dot(basis_funcs, beta))

        # Define the likelihood
        obs = pm.NegativeBinomial("obs", mu=mu, alpha=alpha, observed=Y, dims="obs_idx")

        # Run the sampling using the No-U-Turn Sampler (NUTS) for the specified number of samples
        trace = pm.sample(draws=draws, tune=tune, random_seed=random_seed, target_accept=target_accept)

        return bayesian_model_nb, trace

